In [5]:
import pandas as pd
import numpy as numpy
import scipy.stats

In [6]:
df = pd.DataFrame({'x', 'y'})
print(df)

x_norm = np.linspace(-5,5,300)

x_norm = scipy.stats.norm.rvs(x_norm)

0
0  y
1  x


AttributeError: module 'scipy' has no attribute 'stats'